In [9]:
from sentence_transformers import SentenceTransformer
import requests
import json
import pandas as pd
import numpy as np

In [2]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [3]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [4]:
df = df.iloc[:300]

In [6]:
model_name = 'multi-qa-mpnet-base-dot-v1'

embedding_model = SentenceTransformer(model_name)


You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





model.safetensors:  55%|#####5    | 241M/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
answer_llm = df.iloc[0].answer_llm

In [8]:
embedding_model.encode(answer_llm)[0]

-0.42244643

## Q1: What's the first value of the resulting vector?
### Answer: -0.42

In [10]:
evaluations = []

for _, row in df.iterrows():
    answer_llm_embedding = embedding_model.encode(row.answer_llm)
    answer_orig_embedding = embedding_model.encode(row.answer_orig)
    
    score = np.dot(answer_llm_embedding, answer_orig_embedding)
    evaluations.append(score)

In [11]:
percentile = np.percentile(evaluations, 75)
percentile

31.67430305480957

## Q2. Computing the dot product
Now for each answer pair, let's create embeddings and compute dot product between them
We will put the results (scores) into the evaluations list
What's the 75% percentile of the score?

### Answer: 31.67

In [12]:
def normalize_vector(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

In [13]:
evaluations_normalized = []

for _, row in df.iterrows():
    answer_llm_embedding = embedding_model.encode(row.answer_llm)
    answer_orig_embedding = embedding_model.encode(row.answer_orig)
    
    answer_llm_embedding_norm = normalize_vector(answer_llm_embedding)
    answer_orig_embedding_norm = normalize_vector(answer_orig_embedding)
    
    score = np.dot(answer_llm_embedding_norm, answer_orig_embedding_norm)
    evaluations_normalized.append(score)

In [14]:
percentile_normalized = np.percentile(evaluations_normalized, 75)
percentile_normalized

0.8362346738576889

## Q3. Computing the cosine

From Q2, we can see that the results are not within the [0, 1] range. It's because the vectors coming from this model are not normalized.

So we need to normalize them.

To do it, we

Compute the norm of a vector
Divide each element by this norm
So, for vector v, it'll be v / ||v||

In numpy, this is how you do it:
```
norm = np.sqrt((v * v).sum())
v_norm = v / norm
```

Let's put it into a function and then compute dot product between normalized vectors. This will give us cosine similarity

What's the 75% cosine in the scores?

### Answer: 0.83

In [16]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
from rouge import Rouge
rouge_scorer = Rouge()

r = df.iloc[10]
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
rouge_1_f1 = scores['rouge-1']['f']
rouge_1_f1

0.45454544954545456

## Q4. Rouge

Now we will explore an alternative metric - the ROUGE score.  

This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

It can give a more nuanced view of text similarity than just cosine similarity alone.

We don't need to implement it ourselves, there's a python package for it:

```bash
pip install rouge
```

(The latest version at the moment of writing is `1.0.1`)

Let's compute the ROUGE score between the answers at the index 10 of our dataframe (`doc_id=5170565b`)

```
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
```

There are three scores: `rouge-1`, `rouge-2` and `rouge-l`, and precision, recall and F1 score for each.

* `rouge-1` - the overlap of unigrams,
* `rouge-2` - bigrams,
* `rouge-l` - the longest common subsequence

What's the F score for `rouge-1`?

### Answer: 0.45

In [18]:
avg_rouge = np.mean([scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f']])
avg_rouge

0.35490034990035496

## Q5. Average rouge score

Let's compute the average F-score between `rouge-1`, `rouge-2` and `rouge-l` for the same record from Q4

### Answer: 0.35

In [19]:
rouge_scores = []

for _, row in df.iterrows():
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    
    rouge_1 = scores['rouge-1']['f']
    rouge_2 = scores['rouge-2']['f']
    rouge_l = scores['rouge-l']['f']
    rouge_avg = np.mean([rouge_1, rouge_2, rouge_l])
    
    rouge_scores.append({
        'rouge_1': rouge_1,
        'rouge_2': rouge_2,
        'rouge_l': rouge_l,
        'rouge_avg': rouge_avg
    })

In [21]:
rouge_df = pd.DataFrame(rouge_scores)
avg_rouge_2 = rouge_df['rouge_2'].mean()
avg_rouge_2

0.20696501983423318

## Q6. Average rouge score for all the data points

Now let's compute the score for all the records and create a dataframe from them.

What's the average `rouge_2` across all the records?

### Answer: 0.2